In [60]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

%matplotlib inline

In [7]:
def createCircleMask(height, width, rad):
    # rad: ratio from half of height
    circle = np.zeros((height,width))
    skip_circle = cv2.circle(
        circle,(int(width/2),int(height/2)),
        int(height*.5*rad),1,thickness=-1)
    return circle

In [2]:

## 读取图像，解决imread不能读取中文路径的问题
def cv_imread(filePath):
  try:
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    ## imdecode读取的是rgb，如果后续需要opencv处理的话，需要转换成bgr，转换后图片颜色会变化
    ##cv_img=cv2.cvtColor(cv_img,cv2.COLOR_RGB2BGR)
    return cv_img
  except:
    traceback.print_exc()
    return None

In [132]:

def alignImagesPerspective(im1, im2, mask):
    # Convert images to grayscale
    #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    im1Gray = im1
    im2Gray = im2

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, mask)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, mask)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    #matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imshow("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    points1 = np.expand_dims(points1, 0)
    points2 = np.expand_dims(points2, 0)

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 10)
    assert h is not None
    
    print ('alignImagesPerspective')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [123]:
def alignImagesAffine(im1, im2, mask):
    im1 = im1 * mask
    im2 = im2 * mask
    h = cv2.estimateRigidTransform(im2, im1, False)
    assert h is not None
    h = np.vstack((h, [[0,0,1]]))

    print ('alignImagesAffine:')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [133]:
MAX_FEATURES = 2000
GOOD_MATCH_PERCENT = .5 #0.15

#refFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png"
#imFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154029071.png"
#mask_rad=.4
refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
imFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png"
mask_rad=.2

refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
path = "D:\Images\A70\Image_8419A5/"
mask_rad=.2

mask = 1-createCircleMask(im.shape[0], im.shape[1], mask_rad).astype('uint8')

# Read reference image
print("Reading reference image : ", refFilename)
imReference = cv_imread(refFilename)
imReference = imReference[::4, ::4]
cv2.imshow('ref', imReference)

# Create array of array of images.
imagePaths = []
i = 0        
k = 0
for root, subdirs, files in os.walk(path):
    print("Reading images from " + root + " ... ", flush=True)      
    for filename in files:
        fileExt = os.path.splitext(filename)[1]
        if fileExt not in [".jpg", ".jpeg", ".png"]:
            continue

        imagePath = os.path.join(root, filename)
            
        # Read image to be aligned
        print("Reading image to align : ", imFilename);  
        im = cv_imread(imagePath)
        im = im[::4, ::4]
        cv2.imshow('im', im)

        if im is None :
            print("Error:{} not read properly".format(imagePath))
            continue
        
        #Reference *= mask
        #im *= mask
        
        print("Aligning images ...")
        # Registered image will be resotred in imReg. 
        # The estimated homography will be stored in h. 
        imReg, h = alignImagesPerspective(im, imReference, mask)
        #imReg, h = alignImagesAffine(imReg, imReference, mask)
        
        diff = np.abs(imReg.astype('float')-imReference.astype('float')).astype('uint8')
        cv2.imshow('out', np.hstack([imReference, im, imReg, diff ]))

        # Write aligned image to disk. 
        #outFilename = "aligned.jpg"
        #print("Saving aligned image : ", outFilename); 
        #cv2.imwrite(outFilename, imReg)

        # Print estimated homography
        print("Estimated homography : \n",  h)
        k = cv2.waitKey(0)

        if k==27:
            break
    if k==27:
        break
print ('Done')

Reading reference image :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png
Reading images from D:\Images\A70\Image_8419A5/ ... 
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
alignImagesPerspective
[[ 1.00000000e+00 -1.22617214e-15  9.97458973e-14]
 [-1.09125649e-16  1.00000000e+00  1.07835568e-13]
 [-7.02009342e-19 -5.96720870e-18  1.00000000e+00]]
Estimated homography : 
 [[ 1.00000000e+00 -1.22617214e-15  9.97458973e-14]
 [-1.09125649e-16  1.00000000e+00  1.07835568e-13]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
alignImagesPerspective
[[-7.44923381e-03 -1.02285378e+00  3.56722829e+02]
 [ 1.00409205e+00 -5.51352085e-02 -4.21573450e+01]
 [ 1.02891843e-04 -1.44371081e-04  1.00000000e+00]]
Estimated homography : 
 [[-7.44923381e-03 -1.02285378e+00  3.56722829e+02]
 [ 1.00409205e+00 -5.51352085e-02 -4

Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
alignImagesPerspective
[[-4.68283408e-02  1.04721414e+00  5.09890738e+01]
 [-1.05919447e+00  6.44128289e-02  3.50886490e+02]
 [-3.87859631e-04  3.98574719e-04  1.00000000e+00]]
Estimated homography : 
 [[-4.68283408e-02  1.04721414e+00  5.09890738e+01]
 [-1.05919447e+00  6.44128289e-02  3.50886490e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
alignImagesPerspective
[[ 7.93477511e-02  9.59307183e-01  3.87542202e+01]
 [-9.80352941e-01  2.48596666e-02  3.42461236e+02]
 [ 1.28416528e-04 -1.80172933e-04  1.00000000e+00]]
Estimated homography : 
 [[ 7.93477511e-02  9.59307183e-01  3.87542202e+01]
 [-9.80352941e-01  2.48596666e-02  3.42461236e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_0

In [47]:
cv2.__version__


'3.4.5'